In [64]:
import pandas as pd

# Sample updated data with new start and end values
data = {
    'Category': ['A', 'B', 'A', 'B', 'A'],
    'last': ['aa1', 'bb1', 'aa2', 'bb2', 'cc'],
    'Description': ['first', 'second', 'third', 'fourth', 'fifth'],
    'start': [1, 2, 3, 7, 9],  # 1-based index
    'end': [5, 6, 7, 11, 12]     # 1-based index
}

# Given sequence
sequence = "ATCGATCGATACGATCTGCVCA"

# Create a DataFrame
df = pd.DataFrame(data)

# Identify the grouping key
grouping_key = 'Category'

# Group by the identified key and convert to the desired format
result = {
    category: sorted(
        [
            {**{col: row[col] for col in df.columns if col != grouping_key},
             'substring': sequence[row['start'] - 1: row['end']]}  # Extract substring
            for index, row in group.iterrows()
        ],
        key=lambda x: (x['start'], x['end'])  
    )
    for category, group in df.groupby(grouping_key)
}

# # Initialize a list for the node information

ddf_list = {}
for type in list(result.keys()):
    ddf = pd.DataFrame()

    color_info = [0] * len(sequence)
    color_count = 1
    belongings_info = [set() for i in range(len(sequence))]
    highlight_info = [[i + 1, i + 1] for i in range(len(sequence))]

    type_result = result.get(type, [])
    for entry_idx, entry in enumerate(type_result): # 第几段，段 entry
        start = entry['start'] # start is from 1, not 0
        end = entry['end']
        color_info[start] = color_count
        for _i in range(start, end + 1):
            i = _i - 1
            belongings_info[i].add(entry_idx)
            highlight_info[i][0] = min(highlight_info[i][0], start)
            highlight_info[i][1] = max(highlight_info[i][1], end)
        color_count += 1

    ddf['node'] = [i for i in sequence]
    ddf['color'] = color_info
    ddf['belongings'] = belongings_info
    ddf['highlight_range'] = highlight_info

    ddf_list[type] = ddf

ddf_list['A']


,node,color,belongings,highlight_range
0,A,0,{0},"[1, 5]"
1,T,1,{0},"[1, 5]"
2,C,0,"{0, 1}","[1, 7]"
3,G,2,"{0, 1}","[1, 7]"
4,A,0,"{0, 1}","[1, 7]"
5,T,0,{1},"[3, 7]"
6,C,0,{1},"[3, 7]"
7,G,0,{},"[8, 8]"
8,A,0,{2},"[9, 12]"
9,T,3,{2},"[9, 12]"


In [10]:
import argparse
import configparser

# parser = argparse.ArgumentParser()
# parser.add_argument('--config', type=str, default='/home/platform/project/mrnadesign_platform/mrnadesign_api/workspace/analysis_script/user/1730469766_QLH7/task_prediction_config.ini')

# args = parser.parse_args()
# user_config = args.config

config = configparser.ConfigParser()
config.read('/home/platform/project/mrnadesign_platform/mrnadesign_api/workspace/analysis_script/user/1730469766_QLH7/task_prediction_config.ini')
config.get('ires', 'target_species')


'Homo sapiens'

In [6]:
import pandas as pd
import numpy as np

path = '/home/platform/project/mrnadesign_platform/mrnadesign_api/workspace/mrnadesign_database/'

In [19]:
df1 = pd.read_csv(path + 'Antigen_se.csv')
print(df1.shape)
df2 = pd.read_csv(path + 'TAntigendb_se.csv')
print(df2.shape)
df3 = pd.read_csv(path + 'Tantigen_rna_seq.csv')
print(df3.shape)
df4 = pd.read_csv(path + 'Complete_3_UTR_ARE_Data/Complete_3_UTR_Data.csv')
print(df4.shape)

(505, 20)
(4295, 13)
(4306, 5)
(4884, 10)


In [20]:
df1['Sequence'].to_numpy()

array(['\nMTRNKFIPNKFSIISFSVLLFAISSSQAIEVNAMNEHYTESDIKRNHKTEKNKTEKEKFK\nDSINNLVKTEFTNETLDKIQQTQDLLKKIPKDVLEIYSELGGEIYFTDIDLVEHKELQDL\nSEEEKNSMNSRGEKVPFASRFVFEKKRETPKLIINIKDYAINSEQSKEVYYEIGKGISLD\nIISKDKSLDPEFLNLIKSLSDDSDSSDLLFSQKFKEKLELNNKSIDINFIKENLTEFQHA\nFSLAFSYYFAPDHRTVLELYAPDMFEYMNKLEKGGFEKISESLKKEGVEKDRIDVLKGEK\nALKASGLVPEHADAFKKIARELNTYILFRPVNKLATNLIKSGVATKGLNVHGKSSDWGPV\nAGYIPFDQDLSKKHGQQLAVEKGNLENKKSITEHEGEIGKIPLKLDHLRIEELKENGIIL\nKGKKEIDNGKKYYLLESNNQVYEFRISDENNEVQYKTKEGKITVLGEKFNWRNIEVMAKN\nVEGVLKPLTADYDLFALAPSLTEIKKQIPQKEWDKVVNTPNSLEKQKGVTNLLIKYGIER\nKPDSTKGTLSNWQKQMLDRLNEAVKYTGYTGGDVVNHGTEQDNEEFPEKDNEIFIINPEG\nEFILTKNWEMTGRFIEKNITGKDYLYYFNRSYNKIAPGNKAYIEWTDPITKAKINTIPTS\nAEFIKNLSSIRRSSNVGVYKDSGDKDEFAKKESVKKIAGYLSDYYNSANHIFSQEKKRKI\nSIFRGIQAYNEIENVLKSKQIAPEYKNYFQYLKERITNQVQLLLTHQKSNIEFKLLYKQL\nNFTENETDNFEVFQKIIDEK\n',
       '\nMKKRKVLIPLMALSTILVSSTGNLEVIQAEVKQENRLLNESESSSQGLLGYYFSDLNFQA\nPMVVTSSTTGDLSIPSSELENIPSENQYFQSAIWSGFIKVKKSDEYTFATSADNHVTMWV\nDDQEVINKASNSNKIRLEKGRLYQI

In [15]:
np.unique([df1['Antigen Name'][i] + df1['Antigen Sequence'][i] for i in range(505)]).shape

(485,)

In [10]:
df1['Antigen Name'], df1['Antigen Sequence']

TypeError: 'map' object is not subscriptable

In [21]:
merged_df = pd.merge(df2, df3, how='outer', on=['AgACC'])

merged_df['Antigen Name_x'] = merged_df['Antigen Name_x'].combine_first(merged_df['Antigen Name_y'])
merged_df = merged_df.drop(columns=['Antigen Name_y'])
merged_df.rename(columns={'Antigen Name_x': 'Antigen Name'}, inplace=True)
# merged_df = merged_df.to_numpy()

In [4]:
try:
    float('asd')
except ValueError:
    print('sdf')

sdf


In [26]:
merged_df.shape

(4307, 16)

In [6]:
ddf1 = pd.DataFrame([
    ['a', 1, 11],
    ['b', 2, 22],
    ['c', 3, 33]
], columns=['letter', 'n1', 'n2'])
ddf1

,letter,n1,n2
0,a,1,11
1,b,2,22
2,c,3,33


In [7]:
ddf2 = pd.DataFrame([
    ['a', 111, 1111],
    ['c', 333, 3333],
    ['d', 444, 4444]
], columns=['letter', 'n1', 'n4'])
ddf2

,letter,n1,n4
0,a,111,1111
1,c,333,3333
2,d,444,4444


In [17]:
merged_df = pd.merge(ddf1, ddf2, how='outer', on=['letter'])
merged_df 

,letter,n1_x,n2,n1_y,n4
0,a,1.0,11.0,111.0,1111.0
1,b,2.0,22.0,NaN,NaN
2,c,3.0,33.0,333.0,3333.0
3,d,NaN,NaN,444.0,4444.0


In [18]:
merged_df['n1_x'] = merged_df['n1_x'].combine_first(merged_df['n1_y'])
merged_df = merged_df.drop(columns=['n1_y'])
merged_df.rename(columns={'n1_x': 'n1'}, inplace=True)
merged_df

,letter,n1,n2,n4
0,a,1.0,11.0,1111.0
1,b,2.0,22.0,NaN
2,c,3.0,33.0,3333.0
3,d,444.0,NaN,4444.0
